# Zero-Knowledge Proofs

Zero-knowledge proofs are interactive protocols where a prover convinces a verifier that a statement is true, while revealing nothing beyond the validity of the statement itself. This is a powerful application of the simulation paradigm.

:::{note}
**Reference**: Based on Section 5 of "How To Simulate It" by Yehuda Lindell (2016).
:::

## What is Zero-Knowledge?
:label: zk-intro

In a **zero-knowledge proof**, we have:
- A prover $P$ who knows a witness $w$ for a statement $x \in L$
- A verifier $V$ who wants to be convinced that $x \in L$
- The requirement that $V$ learns **nothing** beyond the fact that $x \in L$

### The Simulation Paradigm for Zero-Knowledge

To prove that a protocol is zero-knowledge, we construct a simulator $\mathcal{S}$ that:
- Receives only the statement $x$ (but NOT the witness $w$)
- Generates a transcript that is indistinguishable from a real interaction between $P(x,w)$ and $V(x)$

If such a simulator exists, it proves that the verifier learns nothing from the interaction, since the same transcript could be generated without the witness.

:::{important} Key Challenge
Unlike in semantic security, the zero-knowledge simulator must produce a convincing transcript without knowing the witness $w$. This typically requires **rewinding** the verifier to extract randomness or adjust the simulation.
:::

## Zero-Knowledge Definition
:label: zk-definition

:::{note} Definition: Zero-Knowledge
Let $(P, V)$ be an interactive proof system for a language $L$. We say that $(P, V)$ is **zero-knowledge** if for every probabilistic polynomial-time verifier $V^*$ there exists a probabilistic polynomial-time simulator $\mathcal{S}$ such that for every $x \in L$ and every witness $w$ for $x$:

$$
\text{VIEW}_{V^*}[P(x,w) \leftrightarrow V^*(x)] \stackrel{c}{\equiv} \mathcal{S}(x)
$$

where $\text{VIEW}_{V^*}$ denotes the view of $V^*$ in the interaction (including its randomness and all messages received).
:::

**Important observations**:
- The simulator $\mathcal{S}$ gets only the statement $x$, not the witness $w$
- This must hold for **every** polynomial-time verifier $V^*$, not just the honest verifier
- We require computational indistinguishability, not perfect indistinguishability

## Example: Graph 3-Coloring
:label: graph-3-coloring

One of the classic zero-knowledge protocols proves knowledge of a 3-coloring of a graph.

### The Problem

- **Input**: A graph $G = (V, E)$ with $n$ vertices
- **Witness**: A valid 3-coloring $\phi: V \rightarrow \{1, 2, 3\}$ such that for all $(u,v) \in E$, $\phi(u) \neq \phi(v)$
- **Goal**: Prove knowledge of $\phi$ without revealing any information about the coloring

### Protocol 5.3: Zero-Knowledge for Graph 3-Coloring

:::{dropdown} **The Protocol**

**Common input**: Graph $G = (V,E)$

**Prover's witness**: Valid 3-coloring $\phi$

**Repeat the following $n|E|$ times**:

1. **Prover commits**:
   - Choose a random permutation $\pi$ of $\{1,2,3\}$
   - For each vertex $v \in V$, commit to $\pi(\phi(v))$ using a commitment scheme
   - Send all $n$ commitments to the verifier

2. **Verifier challenges**:
   - Choose a random edge $(u,v) \in E$
   - Send $(u,v)$ to the prover

3. **Prover responds**:
   - Decommit to the colors of vertices $u$ and $v$ (i.e., reveal $\pi(\phi(u))$ and $\pi(\phi(v))$)

4. **Verifier checks**:
   - Verify that the decommitments are valid
   - Verify that $\pi(\phi(u)) \neq \pi(\phi(v))$
   - Accept if all checks pass
:::

### Why This is Zero-Knowledge

:::{dropdown} **Simulator Construction**

The simulator $\mathcal{S}$ for the honest verifier works as follows:

**Without knowing the coloring $\phi$**:

1. Choose a random edge $(u,v) \in E$ (predicting the verifier's challenge)
2. Choose two random different colors $c_1, c_2 \in \{1,2,3\}$ with $c_1 \neq c_2$
3. Commit to $c_1$ for vertex $u$ and $c_2$ for vertex $v$
4. For all other vertices, commit to random colors
5. When the verifier sends challenge $(u',v')$:
   - If $(u',v') = (u,v)$: proceed with decommitment (colors are already different)
   - If $(u',v') \neq (u,v)$: **rewind** the verifier and try again from step 1

**Analysis**:
- The simulator guesses correctly with probability $1/|E|$
- Expected number of rewinds: $|E|$ (polynomial)
- The simulated transcript is identically distributed to a real execution
- Therefore: $\text{VIEW}_{V}[P(G,\phi) \leftrightarrow V(G)] \equiv \mathcal{S}(G)$
:::

## The Rewinding Technique
:label: rewinding

The key technique in zero-knowledge simulation is **rewinding**:

### How Rewinding Works

1. The simulator runs the verifier and commits to some values
2. The simulator receives the verifier's challenge
3. If the simulator cannot answer this challenge, it:
   - **Rewinds** the verifier to before receiving the challenge
   - Tries again with a different commitment
4. Repeat until finding a commitment that allows answering the challenge

:::{tip} Key Insight
Rewinding is possible in the simulation because the simulator controls the verifier's execution. In a real protocol, the prover cannot rewind the verifier, which is why the prover needs the actual witness to answer any challenge.
:::

### Requirements for Rewinding

For rewinding to be efficient:
- The probability of successfully answering a random challenge must be non-negligible
- The verifier's random choices must be independent across rewinds
- The expected number of rewinds must be polynomial

## Advanced: Non-Black-Box Simulation
:label: non-black-box

The simulation technique described above is **black-box**: the simulator uses the verifier's code as a black box, rewinding it as needed.

### Limitations of Black-Box Simulation

Black-box simulation faces challenges with:
- **Concurrent composition**: When multiple protocol instances run simultaneously, rewinding becomes problematic
- **Auxiliary input**: If the verifier has auxiliary information correlated with the statement, simulation may fail
- **Efficiency**: Rewinding can make simulation inefficient in complex scenarios

### Non-Black-Box Techniques

**Non-black-box simulation** uses the verifier's code directly (not as a black box):
- The simulator can inspect the verifier's program
- Can use techniques like **FLS (Feige-Lapidot-Shamir)** simulation
- Enables stronger security properties like universal composability

## Protocol: Goldreich-Kahan Zero-Knowledge
:label: goldreich-kahan

A more general construction for zero-knowledge proofs for all of NP.

:::{note} Protocol 5.5: Zero-Knowledge for NP

**Common input**: Statement $x$ (claim that $x \in L$ for an NP language $L$)

**Prover's witness**: Witness $w$ such that $(x,w) \in R_L$

**Reduction to Graph 3-Coloring**:
1. Reduce the NP statement to graph 3-coloring (using the Cook-Levin theorem)
2. Run the graph 3-coloring zero-knowledge protocol

This shows that **every NP language has a zero-knowledge proof system**.
:::

## Properties of Zero-Knowledge

### Soundness vs. Zero-Knowledge

Zero-knowledge protocols must balance two properties:

1. **Soundness**: A malicious prover without a witness cannot convince the verifier (except with negligible probability)
2. **Zero-Knowledge**: An honest execution reveals nothing to the verifier beyond the statement's validity

These properties are proven separately:
- Soundness: Show that a cheating prover succeeds with negligible probability
- Zero-knowledge: Construct a simulator that generates indistinguishable transcripts

### Types of Zero-Knowledge

- **Perfect ZK**: Real and simulated views are identically distributed
- **Statistical ZK**: Real and simulated views are statistically close
- **Computational ZK**: Real and simulated views are computationally indistinguishable

Most practical protocols achieve computational zero-knowledge.

## Applications of Zero-Knowledge

Zero-knowledge proofs are fundamental to:

1. **Authentication**: Prove knowledge of a password without revealing it
2. **Blockchain Privacy**: Prove transaction validity without revealing amounts (e.g., zk-SNARKs, zk-STARKs)
3. **Secure Computation**: Component in MPC protocols
4. **Anonymous Credentials**: Prove properties about credentials without revealing the credential itself

:::{important} Key Takeaway
Zero-knowledge proofs demonstrate the power of simulation: by showing that a verifier's view can be simulated without the witness, we prove that the interaction reveals no information beyond the statement's validity. The rewinding technique is essential for constructing such simulators.
:::

## Summary

In this section, we learned:

1. **Zero-knowledge** means the verifier learns nothing beyond the statement's validity
2. Proven via **simulation**: a simulator produces indistinguishable transcripts without the witness
3. **Rewinding** is the key technique allowing simulators to handle verifier challenges
4. **Graph 3-coloring** demonstrates the core ideas with commitment and challenge-response
5. Every **NP language** has a zero-knowledge proof (via reduction to 3-coloring)
6. Zero-knowledge has numerous applications in modern cryptography and privacy-preserving protocols

The simulation paradigm transforms zero-knowledge from an intuitive privacy notion into a rigorously provable cryptographic property.
